<a href="https://colab.research.google.com/github/Anny-tech/Machine-Learning-in-Materials-Science/blob/master/dataset_building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Path to the directory containing the folders

In [3]:
directory = '/content/gdrive/MyDrive/Research/ORNL_research'

In [4]:
import os
import re
import pandas as pd

In [6]:
!pip install -U ovito

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.7/530.7 kB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 MB 7.7 MB/s eta 0:00:00


In [7]:
from ovito.io import import_file
import matplotlib.pyplot as plt
from ovito.data import CutoffNeighborFinder
from ovito.modifiers import ExpressionSelectionModifier, DeleteSelectedModifier
import numpy as np

In [8]:
def cart2pol(X):
    x = X[0]
    y = X[1]
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)


def extract_local_angle(finder, index, central_atom_pos):

        neigh_index = []
        neigh_pos_polar = []
        neigh_pos_cart = []

        for neigh in finder.find(index):
                # print(neigh.index, neigh.distance, neigh.delta)
                neigh_index.append(neigh.index)
                neigh_pos_cart.append(np.array([neigh.delta[0]+central_atom_pos[0], neigh.delta[1]+central_atom_pos[1]]))
                neigh_pos_polar.append(cart2pol(np.array([neigh.delta[0], neigh.delta[1]])))


        neigh_pos_polar = np.array(neigh_pos_polar)
        neigh_pos_cart = np.array(neigh_pos_cart)


        neigh_pos_polar_sorted = neigh_pos_polar[abs(neigh_pos_polar[:,1]).argsort()]
        neigh_pos_cart_sorted = neigh_pos_cart[abs(neigh_pos_polar[:,1]).argsort()]

        neigh_index = np.array(neigh_index)
        neigh_index_sorted = neigh_index[abs(neigh_pos_polar[:,1]).argsort()]

        is_zero = True # In case interested to implement local twists in future

        return {'index' :neigh_index_sorted[0], 'polar_angle': neigh_pos_polar_sorted[0,1], \
                'rel_cart_pos': neigh_pos_cart_sorted[0,:], 'zero_axis': is_zero}


def extract_layer_alignment(finder, initial_index, num_iter=10):
        ic = 0
        lattice_axis_info = {'index': [], 'polar_angle':[], \
                             'rel_cart_pos': [], \
                             'zero_axis': []}
        index = initial_index
        central_atom_pos= np.array([0,0])
        while ic < num_iter:
                local_angle_info = extract_local_angle(finder, index, central_atom_pos)

                # might be smarter way to do the following, for now bootstrap!
                lattice_axis_info['index'].append(local_angle_info['index'])
                lattice_axis_info['polar_angle'].append(local_angle_info['polar_angle'])
                lattice_axis_info['rel_cart_pos'].append(local_angle_info['rel_cart_pos'])
                lattice_axis_info['zero_axis'].append(local_angle_info['zero_axis'])

                index = local_angle_info['index']
                central_atom_pos = local_angle_info['rel_cart_pos']
                print (f'Done with iteration number: {ic}')
                ic+=1

        return lattice_axis_info



def compute_interlayer_angle(layer_1_info, layer_2_info):

        from scipy import stats
        from scipy.stats import t

        x1 = x2 = y1 = y2 = np.array([0.])

        x1=np.append(x1, np.array(layer_1_info['rel_cart_pos'])[:,0][layer_1_info['zero_axis']])
        y1=np.append(y1, np.array(layer_1_info['rel_cart_pos'])[:,1][layer_1_info['zero_axis']])

        x2=np.append(x2, np.array(layer_2_info['rel_cart_pos'])[:,0][layer_2_info['zero_axis']])
        y2=np.append(y2, np.array(layer_2_info['rel_cart_pos'])[:,1][layer_2_info['zero_axis']])

        res1=stats.linregress(x1,y1)
        res2=stats.linregress(x2,y2)

        # plt.plot(x1,y1, 'bo')
        # plt.plot(x1, res1.intercept + res1.slope*x1, 'b-')
        # plt.plot(x2,y2, 'ro')
        # plt.plot(x2, res2.intercept + res2.slope*x2, 'r-')

        tinv = lambda p, df: abs(stats.t.ppf(p/2, df))
        ts1 = tinv(0.05, len(x1)-2)
        err_1 = ts1*res1.stderr

        ts2 = tinv(0.05, len(x2)-2)
        err_2 = ts2*res2.stderr

        num_x = abs(res1.slope -res2.slope)
        denom_y = abs(1 + res1.slope*res2.slope)

        twist_angle = np.arctan2(num_x, denom_y)*180/np.pi

        return twist_angle, err_1, err_2

In [ ]:
# List to store the extracted values
dat = []

# Regular expression to match the folder names
pattern = re.compile(r'(\d+)_([0-9]+[K|M])_rescale_([0-9.]+)_xy_shear_([0-9.]+)')

# Loop through the folders in the directory
for folder_name in os.listdir(directory):
    match = pattern.match(folder_name)
    if match:
        temperature = int(match.group(1))
        temp_scale = match.group(2)
        if temp_scale.endswith('K'):
            temperature *= 1  # Kelvin
        elif temp_scale.endswith('M'):
            temperature *= 1000  # Megakelvin to Kelvin conversion

        strain = float(match.group(3))
        shear_strain = float(match.group(4))

        pipeline = import_file(directory+'/'+folder_name+'/'+f'dump*.dump')
        initial_frame = 100
        timestep = 1000
        cutoff = 4 # for neighbor list building
        id_1 = 163 # any lower layer atom-id
        id_2 = 164 # any upper layer atom-id
        twist_angles = []
        num_frames = pipeline.source.num_frames
        pipeline.modifiers.append(ExpressionSelectionModifier(expression="ParticleType==2"))
        pipeline.modifiers.append(DeleteSelectedModifier())

        for frame in range(initial_frame, num_frames):
            data = pipeline.compute(frame)
            index_1 = np.argwhere(data.particles.identifiers[...]==id_1)
            index_2 = np.argwhere(data.particles.identifiers[...]==id_2)
            finder = CutoffNeighborFinder(cutoff, data)
            layer_1_info = extract_layer_alignment(finder, index_1,num_iter=10)
            layer_2_info = extract_layer_alignment(finder, index_2, num_iter=10)
            twist_angles.append(compute_interlayer_angle(layer_1_info, layer_2_info)[0])

        twist_angle_avg = sum(twist_angles[400:]) / len(twist_angles[400:])
        print(twist_angle_avg)

        dat.append({
            'Folder Name': folder_name,
            'Temperature (K)': temperature,
            'Strain': strain,
            'Shear Strain': shear_strain,
            'Avg_twist_ang': twist_angle_avg
        })

# Create a pandas DataFrame
df = pd.DataFrame(dat)

Streaming output truncated to the last 5000 lines.
Done with iteration number: 0
Done with iteration number: 1
Done with iteration number: 2
Done with iteration number: 3
Done with iteration number: 4
Done with iteration number: 5
Done with iteration number: 6
Done with iteration number: 7
Done with iteration number: 8
Done with iteration number: 9
Done with iteration number: 0
Done with iteration number: 1
Done with iteration number: 2
Done with iteration number: 3
Done with iteration number: 4
Done with iteration number: 5
Done with iteration number: 6
Done with iteration number: 7
Done with iteration number: 8
Done with iteration number: 9
Done with iteration number: 0
Done with iteration number: 1
Done with iteration number: 2
Done with iteration number: 3
Done with iteration number: 4
Done with iteration number: 5
Done with iteration number: 6
Done with iteration number: 7
Done with iteration number: 8
Done with iteration number: 9
Done with iteration number: 0
Done with iteration